## 1\. The World Bank's international debt data

The purpose of this notebook is to analyze a dataset of international debt data collectet by the World Bank. The data contains information about the debt owed by developing countries in various categories, measured in USD. I will be answering questions such as:

\- How much debt is owed in total by the countries in the dataset?

\- Which country has the highest amount of debt and what is that amount?

\- What is the average debt owed by countries different debt indicators

In [1]:
# let's connect to the international_debt database

%%sql
postgresql:///international_debt


'Connected: @international_debt'

In [3]:
# check the column in the database

%%sql

SELECT *
FROM international_debt
LIMIT 10;

 * postgresql:///international_debt
10 rows affected.


country_name,country_code,indicator_name,indicator_code,debt
Afghanistan,AFG,"Disbursements on external debt, long-term (DIS, current US$)",DT.DIS.DLXF.CD,72894453.700000003
Afghanistan,AFG,"Interest payments on external debt, long-term (INT, current US$)",DT.INT.DLXF.CD,53239440.100000001
Afghanistan,AFG,"PPG, bilateral (AMT, current US$)",DT.AMT.BLAT.CD,61739336.899999999
Afghanistan,AFG,"PPG, bilateral (DIS, current US$)",DT.DIS.BLAT.CD,49114729.399999999
Afghanistan,AFG,"PPG, bilateral (INT, current US$)",DT.INT.BLAT.CD,39903620.100000001
Afghanistan,AFG,"PPG, multilateral (AMT, current US$)",DT.AMT.MLAT.CD,39107845
Afghanistan,AFG,"PPG, multilateral (DIS, current US$)",DT.DIS.MLAT.CD,23779724.300000001
Afghanistan,AFG,"PPG, multilateral (INT, current US$)",DT.INT.MLAT.CD,13335820
Afghanistan,AFG,"PPG, official creditors (AMT, current US$)",DT.AMT.OFFT.CD,100847181.900000006
Afghanistan,AFG,"PPG, official creditors (DIS, current US$)",DT.DIS.OFFT.CD,72894453.700000003


## 2\. Finding the number of distinct countries

I can see that the first 10 rows of the table show the amount of debt owed by Afghanistan in different debt indicators. However, I do not know the number of unique countries that are represented in the table. It is likely that some countries appear multiple times because  they have debt in more than one indicator.

In order to accurately perform statistical analysis on the dat, it is important to determine the number of unique countries present in the table. Therefore, I will extract this information in this section.

In [4]:
%%sql
SELECT 
    COUNT(DISTINCT country_name) AS total_distinct_countries
FROM international_debt;

 * postgresql:///international_debt
1 rows affected.


total_distinct_countries
124


## 3\. Finding out the distinct debt indicators

There are a total of 124 countries present on the table. The indicator\_name column specifies the purpose oif the debt. The indicator\_code symbolizes the category of these debts. Knowing about these various debt indicators will help to understand the areas in which a country can possibly be indebted to.

In [5]:
%%sql
SELECT 
    DISTINCT indicator_code AS distinct_debt_indicators
FROM international_debt
ORDER BY distinct_debt_indicators;

 * postgresql:///international_debt
25 rows affected.


distinct_debt_indicators
DT.AMT.BLAT.CD
DT.AMT.DLXF.CD
DT.AMT.DPNG.CD
DT.AMT.MLAT.CD
DT.AMT.OFFT.CD
DT.AMT.PBND.CD
DT.AMT.PCBK.CD
DT.AMT.PROP.CD
DT.AMT.PRVT.CD
DT.DIS.BLAT.CD


## 4\. Totaling the amount of debt owed by the countries

As mentioned earlier, the financial debt of a particular country represents its economic state. But what if I were to project this on an overall global scale?

Let's switch gears from the debt indicators now and find out the total amount of debt (in USD) that is owed by the different countries. This will give a sense of how the overall economy of the entire world is holding up.

In [6]:
%%sql
SELECT 
    ROUND(SUM(debt)/1000000, 2) AS total_debt
FROM international_debt; 

 * postgresql:///international_debt
1 rows affected.


total_debt
3079734.49


## 5\. Country with the highest debt

Now that I have the exact total of the amounts of debt owed by several countries, let's now find out the country that owns the highest amount of debt along with the amount. **Note** that this debt is the sum of different debts owed by a country across several categories. This will help to understand more about the country in terms of its socio-economic scenarios.

In [7]:
%%sql
SELECT 
    country_name, 
    SUM(debt) AS total_debt
FROM international_debt
GROUP BY country_name
ORDER BY total_debt DESC
LIMIT 1;

 * postgresql:///international_debt
1 rows affected.


country_name,total_debt
China,285793494734.200001568


## 6\. Average amount of debt across indicators

So, it was _China_.

I now have a brief overview of the dataset and a few of its summary statistics. I already have an idea of the different debt indicators in which the countries owe their debts. I can dig even further to find out on an average how much debt a country owes? This will give me a better sense of the distribution of the amount of debt across different indicators.

In [8]:
%%sql
SELECT 
    indicator_code AS debt_indicator,
    indicator_name,
    AVG(debt) AS average_debt
FROM international_debt
GROUP BY debt_indicator, indicator_name
ORDER BY average_debt DESC
LIMIT 10;

 * postgresql:///international_debt
10 rows affected.


debt_indicator,indicator_name,average_debt
DT.AMT.DLXF.CD,"Principal repayments on external debt, long-term (AMT, current US$)",5904868401.499193612
DT.AMT.DPNG.CD,"Principal repayments on external debt, private nonguaranteed (PNG) (AMT, current US$)",5161194333.812658349
DT.DIS.DLXF.CD,"Disbursements on external debt, long-term (DIS, current US$)",2152041216.890243888
DT.DIS.OFFT.CD,"PPG, official creditors (DIS, current US$)",1958983452.859836046
DT.AMT.PRVT.CD,"PPG, private creditors (AMT, current US$)",1803694101.963265321
DT.INT.DLXF.CD,"Interest payments on external debt, long-term (INT, current US$)",1644024067.650806481
DT.DIS.BLAT.CD,"PPG, bilateral (DIS, current US$)",1223139290.398230108
DT.INT.DPNG.CD,"Interest payments on external debt, private nonguaranteed (PNG) (INT, current US$)",1220410844.421518983
DT.AMT.OFFT.CD,"PPG, official creditors (AMT, current US$)",1191187963.083064523
DT.AMT.PBND.CD,"PPG, bonds (AMT, current US$)",1082623947.653623188


## 7\. The highest amount of principal repayments

The indicator `DT.AMT.DLXF.CD` tops the chart of average debt. This category includes repayment of long term debts. Countries take on long-term debt to acquire immediate capital.

An interesting observation in the above finding is that there is a huge difference in the amounts of the indicators after the second one. This indicates that the first two indicators might be the most severe categories in which the countries owe their debts.

Let's investigate this a bit more so as to find out which country owes the highest amount of debt in the category of long term debts (`DT.AMT.DLXF.CD`). Since not all the countries suffer from the same kind of economic disturbances, this finding will allow me to understand that particular country's economic condition a bit more specifically.

In [9]:
%%sql
SELECT 
    country_name, 
    indicator_name
FROM international_debt
WHERE debt = (SELECT 
                  MAX(debt)
              FROM international_debt
              WHERE indicator_code='DT.AMT.DLXF.CD');

 * postgresql:///international_debt
1 rows affected.


country_name,indicator_name
China,"Principal repayments on external debt, long-term (AMT, current US$)"


## 8\. The most common debt indicator

China has the highest amount of debt in the long-term debt (`DT.AMT.DLXF.CD`) category. 

The long-term debt is the topmost category when it comes to the average amount of debt. But is it the most common indicator in which the countries owe their debt? Let's find that out.

In [10]:
%%sql
SELECT 
    indicator_code,
    COUNT(indicator_code) AS indicator_count
FROM international_debt
GROUP BY indicator_code
ORDER BY indicator_count DESC, indicator_code DESC
LIMIT 20;


 * postgresql:///international_debt
20 rows affected.


indicator_code,indicator_count
DT.INT.OFFT.CD,124
DT.INT.MLAT.CD,124
DT.INT.DLXF.CD,124
DT.AMT.OFFT.CD,124
DT.AMT.MLAT.CD,124
DT.AMT.DLXF.CD,124
DT.DIS.DLXF.CD,123
DT.INT.BLAT.CD,122
DT.DIS.OFFT.CD,122
DT.AMT.BLAT.CD,122


## 9\. Other viable debt issues and conclusion

There are a total of six debt indicators in which all the countries listed in our dataset have taken debt. The indicator `DT.AMT.DLXF.CD` is also there in the list. So, this gives a clue that all these countries are suffering from a common economic issue. But that is not the end of the story, but just a part of the story.

Let's change tracks from `debt_indicator`s now and focus on the amount of debt again. Let's find out the maximum amount of debt that each country has. With this, we will be in a position to identify the other plausible economic issues a country might be going through.

In this notebook, I took a look at debt owed by countries across the globe. I extracted a few summary statistics from the data and unraveled some interesting facts and figures. I also validated my findings to make sure the investigations are correct.

In [11]:
%%sql
SELECT 
    country_name, 
    MAX(debt) AS maximum_debt
FROM international_debt
GROUP BY country_name
ORDER BY maximum_debt DESC
LIMIT 10;

 * postgresql:///international_debt
10 rows affected.


country_name,maximum_debt
China,96218620835.699996948
Brazil,90041840304.100006104
Russian Federation,66589761833.5
Turkey,51555031005.800003052
South Asia,48756295898.199996948
Least developed countries: UN classification,40160766261.599998474
IDA only,34531188113.199996948
India,31923507000.799999237
Indonesia,30916112653.799999237
Kazakhstan,27482093686.400001526
